In [167]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib as  mpl

In [168]:
mpl.rcParams[u'font.sans-serif'] = ['simhei']
mpl.rcParams['axes.unicode_minus'] = False

In [169]:
def readData(y,s,c):
    df = pd.read_csv("lvr_land/"+ y +"-s" + s + "/" + c + "_lvr_land_A.csv",encoding='utf-8',low_memory=False)
    return df

In [170]:
def combineDF(df1,df2):
    df = pd.concat([df1,df2],axis = 0)
    return df

In [171]:
def selectFarmland(df):
    
    df = df[["鄉鎮市區","土地移轉總面積(平方公尺)","交易標的","都市土地使用分區","非都市土地使用分區","非都市土地使用編定","總價(元)","單價(元/平方公尺)"]]
    df_farmland = df[((df.交易標的 == "土地")&(((df.都市土地使用分區 =="農")&(df.非都市土地使用編定 !="丁種建築用地"))| ((df.非都市土地使用分區 == "一般農業區")& (df.非都市土地使用編定 !="丁種建築用地"))|((df.非都市土地使用分區 == '特定農業區')& (df.非都市土地使用編定 !="丁種建築用地")))) ]
    
    return df_farmland

In [172]:
def fillNanprice(df_farmland):
    #modify zero or nan price
    df_ok = df_farmland[(df_farmland["單價(元/平方公尺)"].notnull())&(df_farmland["單價(元/平方公尺)"] != "0")]
    df = df_farmland[(df_farmland["單價(元/平方公尺)"].isnull())|(df_farmland["單價(元/平方公尺)"] == "0")]
    df = df[(df['土地移轉總面積(平方公尺)'] != "0")&(df['總價(元)'] != "0")]
    df["單價(元/平方公尺)"] = calculateprice(df['總價(元)'],df['土地移轉總面積(平方公尺)'])
    df_farmland = pd.concat([df_ok,df],axis = 0)
    return df_farmland

In [173]:
def calculateprice(total_price,area):
    return pd.to_numeric(total_price)/pd.to_numeric(area)

In [174]:
def calculateMean(df_farmland):

    landpriceMean = pd.to_numeric(df_farmland['單價(元/平方公尺)']).mean()
   
    return landpriceMean

In [175]:
def calculateGain(cityMeandict,div):
    
    #dict to df
    arr = list(cityMeandict.items())
    df = pd.DataFrame(arr, columns=['年份','單價'])
    
    #  #逐年漲幅
    gain = df["單價"].pct_change(div)
    
    return gain

In [176]:
def grouptown_calculateMean(df,list_all_town):
    
    grouped=pd.to_numeric(df['單價(元/平方公尺)']).groupby(df['鄉鎮市區'])
    town_dict = {}
    for key,group in grouped:  
        m = grouped.mean()
        town_dict[key]= m[key] 
        if key not in list_all_town:
            list_all_town.append(key)
    return {"towndict":town_dict,"town_list":list_all_town}

In [177]:
def plottown_average(towndict,list_all_town):
    list_color = ['#393b79','#5254a3','#6b6ecf','#9c9ede','#637939','#8ca252','#b5cf6b','#cedb9c','#8c6d31','#bd9e39','#e7ba52','#e7cb94','#843c39','#ad494a','#d6616b','#e7969c','#7b4173','#a55194','#ce6dbd','#de9ed6','#3182bd','#6baed6','#9ecae1','#c6dbef','#e6550d','#fd8d3c','#fdae6b','#fdd0a2','#e6550d','#fd8d3c','#fdae6b','#fdd0a2','#636363','#969696','#bdbdbd','#d9d9d9']
    color = 0
    for town in list_all_town: 
        town_year_dict = {}
        for key in towndict:   
            if town in towndict[key]:
                town_year_dict[key] = towndict[key][town]
        
        plt.plot(town_year_dict.keys(),town_year_dict.values(),marker='o',linestyle='solid',label=town if(town!='fa72埔鄉')else'鹽埔鄉')
        
        plt.xticks(range(len(towndict)), towndict.keys(),rotation=90) 
        plt.xlabel("年")
        plt.ylabel("(元/平方公尺)")
        plt.legend(loc=2, bbox_to_anchor=(1.05,1.0),borderaxespad = 0.)
        plt.subplots_adjust(right=0.7)
        plt.title(list_city[i] + " "+"各鄉鎮市區"+"單價")
        plt.grid()
        plt.savefig("town-"+list_city[i])
        plt.show()
        color = color + 1
        
        for i in range(101,108):
            print("|"+str(i)+"|",end='')
            if np.isnan(town_year_dict[i]) == False:
                print(str(town_year_dict[i])+"|") 
            else:
                print("-|")

In [184]:
#list_cityfilename = ['A','B','C','D','E','F','G','H','I','J','K','M','N','O','P','Q','T','U','V','W','X','Z']
#list_city = ['台北市','台中市','基隆市','台南市','高雄市','新北市','宜蘭縣','桃園縣','嘉義市','新竹縣','苗栗縣','南投縣','彰化縣','新竹市','雲林縣','嘉義縣','屏東縣','花蓮縣','台東縣','金門縣','澎湖縣','連江縣']
list_city  = ['花蓮縣','苗栗縣','新竹縣','台東縣','屏東縣',]
list_cityfilename = ['U','K','J','V','T']
list_color = ['#393b79','#5254a3','#6b6ecf','#9c9ede','#637939','#8ca252','#b5cf6b','#cedb9c','#8c6d31','#bd9e39','#e7ba52','#e7cb94','#843c39','#ad494a','#d6616b','#e7969c','#7b4173','#a55194','#ce6dbd','#de9ed6','#3182bd','#6baed6','#9ecae1','#c6dbef','#e6550d','#fd8d3c','#fdae6b','#fdd0a2']
i=0
taiwan = {} #各縣市逐年平均
city_gain = {} #縣市逐年漲幅
全國縣市單價 = {}
for c in list_cityfilename: #every city
    
    s = 4 # from 101-s4 to 107-s2
    cityMeandict = {} #各城市逐年平均
    townMeandict = {} #各鄉鎮逐年平均
    list_all_town = [] #該城市的鄉鎮列表
    for y in range(101,108): #every year
        df = pd.DataFrame()
        while s <= 4:
            # read data
            df_s =  readData(str(y),str(s),c)
            df = combineDF(df,df_s)

            if y == 107 and s == 2:
                break
            s = s+1
        s=1
        # select farmland
        df_farmland = selectFarmland(df)
        # fill nan price
        df_farmland = fillNanprice(df_farmland)
        
        # 縣市年的平均
        mean = calculateMean(df_farmland)
        cityMeandict[str(y)] = mean
       
        # 鄉鎮
        result = grouptown_calculateMean(df_farmland,list_all_town)
        townMeandict[str(y)] = result["towndict"]
    全國縣市單價[list_city[i]] = cityMeandict
    
    #各鄉鎮逐年單價排名
    print("各鄉鎮逐年單價排名")
    table_dict = {}
    for y in range(101,108):
        sorted_town = sorted(townMeandict[str(y)].items(),key = lambda s:s[1],reverse=True)
        table_dict[y] = sorted_town
   
    for t in range(len(list_all_town)):
        print("|"+str(t+1)+"|",end='')
        for y in range(101,108):
            if t < len(table_dict[y]):
                print(table_dict[y][t][0]+"/"+"{:.1f}".format(table_dict[y][t][1])+"|",end="") 
            else:
                print("-|",end="")
        print("")
        
    #plottown_average(townMeandict,list_all_town)
    taiwan[list_city[i]] = cityMeandict
  
    #鄉鎮逐年漲幅
    print("鄉鎮逐年漲幅")
    鄉鎮逐年平均單價={}
    鄉鎮歷年平均單價 = {}
    鄉鎮年度漲幅 = {}
    鄉鎮逐年漲幅 = {}
    for t in list_all_town:
        鄉鎮逐年平均單價 = {}
        for y in range(101,108):
            if t in townMeandict[str(y)].keys():
                鄉鎮逐年平均單價[y] = townMeandict[str(y)][t]
        鄉鎮歷年平均單價[t] = 鄉鎮逐年平均單價
      
        gain = calculateGain(鄉鎮歷年平均單價[t],1)
        year_list = list(鄉鎮歷年平均單價[t].keys())
        
        y=0
        鄉鎮逐年漲幅 = {}
        for g in gain:
            鄉鎮逐年漲幅[year_list[y]] = g
            y+=1
        鄉鎮年度漲幅[t] = 鄉鎮逐年漲幅
    
    for_sort = {}
    各年漲幅 = {}
    sorted_gain = {}
    for y in range(102,108):
        for_sort = {}
        for t in list_all_town:
            if y in 鄉鎮年度漲幅[t].keys():
                for_sort[t] = 鄉鎮年度漲幅[t][y]
        各年漲幅[y] = for_sort

    for y in range(102,108):
        sorted_gain[y] = sorted(各年漲幅[y].items(),key=lambda s:s[1],reverse=True)
    
    sorted_dict = {}
    for y in range(102,108):
        sorted_dict[y] = dict(sorted_gain[y])
        
    for s in list_all_town:
        print("|"+str(s),end="")
    print("|")
    for s in list_all_town:
        print("|------",end="")
    print("|")
    
    for y in range(102,108):
        print("|"+str(y)+"|",end='')
        for t in list_all_town:
            if t in sorted_dict[y].keys():
                print("{:.1f}".format(sorted_dict[y][t]*100)+"|",end='')  
            else:
                print("-|",end='')
        print("")
        
    #鄉鎮總漲幅
    print("鄉鎮總漲幅")
    鄉鎮逐年平均單價={}
    鄉鎮歷年平均單價 = {}
    鄉鎮年度漲幅 = {}
    鄉鎮逐年漲幅 = {}
    for t in list_all_town:
        鄉鎮逐年平均單價 = {}
        for y in range(101,108):
            if t in townMeandict[str(y)].keys():
                鄉鎮逐年平均單價[y] = townMeandict[str(y)][t]
        鄉鎮歷年平均單價[t] = 鄉鎮逐年平均單價
      
        gain = calculateGain(鄉鎮歷年平均單價[t],len(鄉鎮歷年平均單價[t].keys())-1)
        for g in gain:
            if np.isnan(g) == False:
                 鄉鎮年度漲幅[t] = g
    
    sorted_gain = sorted(鄉鎮年度漲幅.items(),key=lambda s:s[1],reverse=True)

    for h in range(len(鄉鎮年度漲幅.keys())):
        print("|"+str(h)+"|",end="")  
        print(sorted_gain[h][0]+"/"+"{:.1f}".format(sorted_gain[h][1]*100)+"|")     
      
    i = i+1
    

各鄉鎮逐年單價排名
|1|花蓮市/6180.5|花蓮市/8725.8|花蓮市/7873.7|花蓮市/10393.9|新城鄉/14739.7|新城鄉/25073.7|花蓮市/20455.2|
|2|吉安鄉/3600.9|吉安鄉/5280.6|新城鄉/5836.8|吉安鄉/6419.9|花蓮市/13070.6|花蓮市/11899.8|新城鄉/17963.9|
|3|新城鄉/2236.8|新城鄉/4912.6|吉安鄉/4937.2|新城鄉/5682.8|吉安鄉/7056.8|吉安鄉/8617.6|吉安鄉/8254.2|
|4|豐濱鄉/1502.5|壽豐鄉/2171.5|壽豐鄉/2807.3|壽豐鄉/3089.7|壽豐鄉/4043.1|豐濱鄉/4440.5|鳳林鎮/4394.5|
|5|壽豐鄉/1478.9|豐濱鄉/1900.4|瑞穗鄉/1846.7|豐濱鄉/2694.2|鳳林鎮/2936.2|壽豐鄉/4354.9|壽豐鄉/2794.4|
|6|瑞穗鄉/1211.4|鳳林鎮/1431.6|豐濱鄉/1588.1|鳳林鎮/2495.5|瑞穗鄉/2008.0|鳳林鎮/2133.8|瑞穗鄉/2064.8|
|7|秀林鄉/974.8|瑞穗鄉/1285.1|鳳林鎮/1366.9|瑞穗鄉/2161.4|豐濱鄉/1898.8|瑞穗鄉/1985.8|豐濱鄉/1332.0|
|8|鳳林鎮/963.2|玉里鎮/990.7|玉里鎮/1024.8|富里鄉/1401.4|秀林鄉/1484.2|富里鄉/1798.0|秀林鄉/1151.8|
|9|玉里鎮/731.5|富里鄉/890.0|光復鄉/815.4|玉里鎮/1384.2|富里鄉/1329.7|光復鄉/1247.0|玉里鎮/1090.8|
|10|光復鄉/441.1|光復鄉/658.6|富里鄉/790.4|光復鄉/1134.5|光復鄉/1298.9|玉里鎮/1153.9|富里鄉/953.2|
|11|富里鄉/355.9|秀林鄉/592.3|秀林鄉/679.1|秀林鄉/919.1|玉里鎮/1117.8|秀林鄉/877.2|光復鄉/807.2|
|12|萬榮鄉/355.0|卓溪鄉/177.2|萬榮鄉/360.1|萬榮鄉/498.5|萬榮鄉/334.7|萬榮鄉/535.4|萬榮鄉/415.6|
|13|卓溪鄉/135.2|萬榮鄉/174.7|-|-|卓溪鄉

|23|里港鄉/4.5|
|24|竹田鄉/4.4|
|25|枋山鄉/2.5|
|26|泰武鄉/0.0|
|27|霧台鄉/0.0|
|28|牡丹鄉/-27.9|
|29|三地門鄉/-32.3|
|30|滿州鄉/-45.7|
|31|南州鄉/-54.0|


In [188]:
#縣市各年漲幅
縣市逐年漲幅={}
for t in list_city:
    gain = calculateGain(全國縣市單價[t],1)#總漲幅 argu2 = 6
    y=101
    年度漲幅 = {}
    for g in gain:
        年度漲幅[y] = g
        y=y+1
    縣市逐年漲幅[t] = 年度漲幅
for_sort = {}
各年漲幅 = {}
sorted_gain = {}
for y in range(102,108):
    for_sort = {}
    for t in list_city:
        for_sort[t] = 縣市逐年漲幅[t][y]
    各年漲幅[y] = for_sort

for y in range(102,108):
    sorted_gain[y] = sorted(各年漲幅[y].items(),key=lambda s:s[1],reverse=True)
for y in range(102,108):
    print("|"+str(y),end="")
print("|")
for y in range(102,108):
    print("|------",end="")
print("|")

for i in range(0,5):
    print("|"+str(i),end="")
    for y in range(102,108):
        print("|"+sorted_gain[y][i][0]+"/"+ "{:.1f}".format(sorted_gain[y][i][1]*100),end="")
    print("|")

|102|103|104|105|106|107|
|------|------|------|------|------|------|
|0|花蓮縣/64.9|苗栗縣/43.2|台東縣/35.6|苗栗縣/35.4|花蓮縣/38.0|苗栗縣/14.1|
|1|苗栗縣/46.6|新竹縣/37.8|花蓮縣/20.9|台東縣/16.8|屏東縣/11.9|新竹縣/5.6|
|2|新竹縣/43.9|台東縣/8.9|屏東縣/12.5|花蓮縣/13.3|新竹縣/-11.2|屏東縣/-0.3|
|3|台東縣/25.8|屏東縣/7.9|新竹縣/4.8|屏東縣/8.6|台東縣/-12.2|台東縣/-7.9|
|4|屏東縣/8.0|花蓮縣/0.9|苗栗縣/-15.5|新竹縣/-0.2|苗栗縣/-20.1|花蓮縣/-10.0|
